<a href="https://colab.research.google.com/github/mavillan/jane-street-market-prediction/blob/main/tuning/snn-optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install -Iv scikit-learn==0.23.2 > /dev/null 2>&1
!pip install optuna > /dev/null 2>&1

In [3]:
import copy
import os
import numpy as np 
import pandas as pd
from pathlib import Path
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import optuna

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

import torch
import torch.nn as nn
from torch.utils.data import Dataset,TensorDataset,DataLoader

# custom modules
import sys
sys.path.append("/content/drive/MyDrive/kaggle/janestreet")
from torch_utils import Monitor, train_step, valid_step

def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    
set_seed(2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

***
### utility functions

In [4]:
def utility_score(date, weight, resp, action):
    """
    Fast computation of utility score
    """
    date = date.astype(int)
    count_i = len(np.unique(date))
    Pi = np.bincount(date, weight * resp * action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(Pi)
    return -u

In [5]:
def cat_encoder(X):
    """
    Fast one-hot encoding of feature_0
    """
    X["feature_00"] = 0
    idx00 = X.query("feature_0 == -1").index
    X.loc[idx00,"feature_00"] = 1
    
    X["feature_01"] = 0
    idx01 = X.query("feature_0 == 1").index
    X.loc[idx01,"feature_01"] = 1
    
    return X.iloc[:,1:]

In [6]:
def show_metrics(monitor):
    x = np.arange(len(monitor.train_loss))
    
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(21, 7))
    
    ax1 = axes[0]
    ax2 = ax1.twinx()
    ax1.plot(x, monitor.train_loss, 'go-', label="train_loss")
    ax2.plot(x, monitor.train_metric, 'ro-', label="train_metric")
    plt.legend(loc="best")
    ax1.set_xlabel('epochs')
    ax1.set_ylabel('loss')
    ax1.set_title("Training")
    plt.grid()
    
    ax1 = axes[1]
    ax2 = ax1.twinx()
    ax1.plot(x, monitor.valid_loss, 'go-', label="valid_loss")
    ax2.plot(x, monitor.valid_metric, 'ro-', label="valid_metric")
    plt.legend(loc="best")
    ax1.set_xlabel('epochs')
    ax2.set_ylabel('metric')
    ax1.set_title("Validation")
    plt.grid()
    
    plt.show()

***
### preparing the data

In [7]:
root = Path("/content/drive/MyDrive/kaggle/janestreet/preprocessing/")

train = pd.read_parquet(root/"train.parquet")
features = pd.read_parquet(root/"features.parquet")

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2390491 entries, 0 to 2390490
Columns: 143 entries, date to w4
dtypes: float32(140), int16(1), int32(1), int8(1)
memory usage: 1.3 GB


In [8]:
train = train.query("date > 85").query("weight > 0").reset_index(drop=True)

input_features = [col for col in train.columns if "feature" in col]
resp_cols = ['resp', 'resp_1', 'resp_2', 'resp_3', 'resp_4']
w_cols = ["w", "w1", "w2", "w3", "w4"]

X_dset = train.loc[:,input_features].copy()
y_dset = (train.loc[:,resp_cols] > 0).astype(int).copy()
w_dset = train.loc[:, w_cols].copy()
dwr_dset = train.loc[:, ["date","weight","resp"]].copy()

In [9]:
%%time 

with open("/content/drive/MyDrive/kaggle/janestreet/imputer/imputer_f0m1.pickle", "rb") as file:
    imputer_f0m1 = pickle.load(file)
    file.close()
    
with open("/content/drive/MyDrive/kaggle/janestreet/imputer/imputer_f0p1.pickle", "rb") as file:
    imputer_f0p1 = pickle.load(file)
    file.close()

idx_f0m1 = X_dset.query("feature_0 == -1").index
X_dset.loc[idx_f0m1, input_features[1:]] = imputer_f0m1.transform(X_dset.loc[idx_f0m1, input_features[1:]])

idx_f0p1 = X_dset.query("feature_0 ==  1").index
X_dset.loc[idx_f0p1, input_features[1:]] = imputer_f0p1.transform(X_dset.loc[idx_f0p1, input_features[1:]])

CPU times: user 41.8 s, sys: 6.44 s, total: 48.3 s
Wall time: 44.4 s


In [10]:
X_dset = cat_encoder(X_dset)
input_features = X_dset.columns.tolist()

***
### model definition

In [11]:
class GBN(nn.Module):
    """
    Ghost Batch Normalization
    https://arxiv.org/abs/1705.08741
    """

    def __init__(self, input_dim, virtual_batch_size=128, momentum=0.01):
        super(GBN, self).__init__()

        self.input_dim = input_dim
        self.virtual_batch_size = virtual_batch_size
        self.bn = nn.BatchNorm1d(self.input_dim, momentum=momentum)

    def forward(self, x):
        chunks = x.chunk(int(np.ceil(x.shape[0] / self.virtual_batch_size)), 0)
        res = [self.bn(x_) for x_ in chunks]

        return torch.cat(res, dim=0)

class NormalLinear(nn.Module):
    """ 
    Linear layer with normalized weights
    """
    def __init__(self, size_in, size_out, bias=True):
        super().__init__()
        self.size_in, self.size_out = size_in, size_out
        # weights vector
        weights_v = torch.Tensor(size_out, size_in)
        nn.init.kaiming_uniform_(weights_v, a=np.sqrt(5)) 
        self.weights_v = nn.Parameter(weights_v)
        # weights magnitude
        weights_m = torch.norm(weights_v, dim=1, keepdim=True)
        self.weights_m = nn.Parameter(weights_m)
        
        if bias:
            bias_v = torch.Tensor(size_out)    
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(weights_v)
            bound = 1 / np.sqrt(fan_in)
            nn.init.uniform_(bias_v, -bound, bound)
            self.bias = nn.Parameter(bias_v)
        else:
            self.register_parameter('bias', None)
        
        self._normalize_weights()
            
    def _normalize_weights(self):
        with torch.set_grad_enabled(False):
            norm_per_output = torch.norm(self.weights_v, dim=1, keepdim=True)
            self.weights_v.div_(norm_per_output)
            
    def forward(self, x):
        self._normalize_weights()
        return nn.functional.linear(x, self.weights_v * self.weights_m, self.bias)

In [12]:
class SNN(nn.Module):
    """
    SNN for pretraining
    """

    def __init__(self, input_dim, output_dim, nn_depth, nn_width, dropout, momentum=0.02, virtual_batch_size=128):
        super().__init__()
        
        self.bn_in = GBN(input_dim, virtual_batch_size=virtual_batch_size, momentum=momentum)
        self.dp_in = nn.Dropout(dropout)
        self.ln_in = NormalLinear(input_dim, nn_width, bias=False)
        
        self.bnorms = nn.ModuleList(
            [GBN(nn_width, virtual_batch_size=virtual_batch_size, momentum=momentum) 
             for i in range(nn_depth-1)])
        self.dropouts = nn.ModuleList(
            [nn.Dropout(dropout) 
             for i in range(nn_depth-1)])
        self.linears = nn.ModuleList(
            [NormalLinear(nn_width, nn_width, bias=False) 
             for i in range(nn_depth-1)])
        
        self.bn_out = GBN(nn_width, virtual_batch_size=virtual_batch_size, momentum=momentum)
        self.dp_out = nn.Dropout(dropout/2)
        self.ln_out = NormalLinear(nn_width, output_dim, bias=False)

    def forward(self, x):
        x = self.bn_in(x)
        x = self.dp_in(x)
        x = self.ln_in(x)
        x = nn.functional.relu(x)

        for bn_layer,dp_layer,ln_layer in zip(self.bnorms,self.dropouts,self.linears):
            x = bn_layer(x)
            x = dp_layer(x)
            x = ln_layer(x)
            x = nn.functional.relu(x)
            
        x = self.bn_out(x)
        x = self.dp_out(x)
        x = self.ln_out(x)
        return x

In [13]:
class BCELabelSmoothing(nn.Module):
    def __init__(self, label_smoothing=0.0):
        super(BCELabelSmoothing, self).__init__()
        self.label_smoothing = label_smoothing
        self.bce_loss = torch.nn.functional.binary_cross_entropy_with_logits
        
    def forward(self, prediction, target, weight=None):
        target_smooth = target*(1.0 - self.label_smoothing) + 0.5*self.label_smoothing
        if weight is None:
            loss = self.bce_loss(prediction, target_smooth, reduction="mean")
        else:
            loss = self.bce_loss(prediction, target_smooth, weight, reduction="sum") / torch.sum(weight)
        return loss

bce_loss = BCELabelSmoothing(label_smoothing=1e-2)

***
### Hyperparameter tuning

In [14]:
# 80% rows for train & 20% for valid
train_idx = train.query("date < 430").index
valid_idx = train.query("date >= 430").index

train_dset = TensorDataset(torch.tensor(X_dset.loc[train_idx].values, dtype=torch.float), 
                           torch.tensor(y_dset.loc[train_idx].values, dtype=torch.float),
                           torch.tensor(w_dset.loc[train_idx].values, dtype=torch.float),
                           torch.tensor(dwr_dset.loc[train_idx].values, dtype=torch.float),
                          )

valid_dset = TensorDataset(torch.tensor(X_dset.loc[valid_idx].values, dtype=torch.float), 
                           torch.tensor(y_dset.loc[valid_idx].values, dtype=torch.float),
                           torch.tensor(w_dset.loc[valid_idx].values, dtype=torch.float),
                           torch.tensor(dwr_dset.loc[valid_idx].values, dtype=torch.float),
                          )

dataset_sizes = {'train': len(train_dset), 'valid': len(valid_dset)}
train_dataloader = DataLoader(train_dset, batch_size=2048, shuffle=True, num_workers=2)
valid_dataloader = DataLoader(valid_dset, batch_size=len(valid_dset), shuffle=False, num_workers=2)

print("Number of step per epoch:", len(train_dset)//2048)

Number of step per epoch: 612


In [15]:
if os.path.exists(f"/content/drive/MyDrive/kaggle/janestreet/logs/snn_tuning.csv"):
    logger = open(f"/content/drive/MyDrive/kaggle/janestreet/logs/snn_tuning.csv", "a")
else:
    logger = open(f"/content/drive/MyDrive/kaggle/janestreet/logs/snn_tuning.csv", "w")
    logger.write("trial;params;loss;metric;loss_hist;metric_hist\n")

In [16]:
default_nn_kwargs = {
    "input_dim":len(input_features),
    "output_dim":len(resp_cols),
    "nn_depth":3,
    }

def objective(trial):
    sampled_nn_kwargs = {
        #"nn_depth": trial.suggest_int("nn_depth", 3, 5)
        "nn_width": int(trial.suggest_discrete_uniform("nn_width", 64, 192, 16)),
        "dropout": trial.suggest_discrete_uniform("nn_dropout", 0.1, 0.5, 0.05),
        "momentum": trial.suggest_discrete_uniform("momentum", 0.01, 0.1, 0.01),
        "virtual_batch_size": 2 ** trial.suggest_int("virtual_batch_size", 7, 10),
        }
    nn_kwargs = {**sampled_nn_kwargs, **default_nn_kwargs}

    # other hyperparams
    weight_decay = 10 ** trial.suggest_int("weight_decay", -6, -2)
    pct_start = trial.suggest_discrete_uniform("pct_start", 0.1, 0.5, 0.1)

    sampled_params = {
        **sampled_nn_kwargs,
        "weight_decay":weight_decay,
        "pct_start":pct_start,
        }
    print("-"*80)
    print("sampled_params:", sampled_params)

    model = SNN(**nn_kwargs)
    model = model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=5e-2, momentum=0.9, weight_decay=weight_decay)

    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, 
        max_lr=1e-1,
        epochs=50,
        pct_start=pct_start, 
        anneal_strategy='cos', 
        cycle_momentum=True, 
        base_momentum=0.8, 
        max_momentum=0.9, 
        div_factor=1e1,
        final_div_factor=1e0,
        steps_per_epoch=len(train_dataloader),
        verbose=False)
    
    monitor = Monitor(
        model=model,
        optimizer=optimizer,
        scheduler=scheduler,
        patience=10,
        metric_fn=utility_score,
        experiment_name=f'SNN',
        num_epochs=50,
        dataset_sizes=dataset_sizes,
        early_stop_on_metric=False,
        lower_is_better=True)
    
    for epoch in monitor.iter_epochs:
        train_step(model, train_dataloader, optimizer, monitor, bce_loss, scheduler=scheduler, clip_value=None)    
        early_stop = valid_step(model, valid_dataloader, optimizer, monitor, bce_loss)
        if early_stop: break

    logger.write(f"{trial.number};{sampled_params};{monitor.best_loss};{monitor.best_metric};{monitor.valid_loss[-10:]};{monitor.valid_metric[-10:]}\n")
    logger.flush()

    print(f"best_valid_loss: {monitor.best_loss} - best_valid_metric: {monitor.best_metric}")
    return monitor.best_loss

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200, timeout=43200, show_progress_bar=False)

[I 2021-02-16 18:55:36,220] A new study created in memory with name: no-name-c432670e-9f86-4f7e-855e-d256c09fa035


--------------------------------------------------------------------------------
sampled_params: {'nn_width': 176, 'dropout': 0.30000000000000004, 'momentum': 0.05, 'virtual_batch_size': 128, 'weight_decay': 0.0001, 'pct_start': 0.4}


[I 2021-02-16 19:28:31,940] Trial 0 finished with value: 0.6856903433799744 and parameters: {'nn_width': 176.0, 'nn_dropout': 0.30000000000000004, 'momentum': 0.05, 'virtual_batch_size': 7, 'weight_decay': -4, 'pct_start': 0.4}. Best is trial 0 with value: 0.6856903433799744.



best_valid_loss: 0.6856903433799744 - best_valid_metric: -2548.4040877590623
--------------------------------------------------------------------------------
sampled_params: {'nn_width': 160, 'dropout': 0.45000000000000007, 'momentum': 0.08, 'virtual_batch_size': 1024, 'weight_decay': 0.01, 'pct_start': 0.5}


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: RuntimeWarning:

invalid value encountered in double_scalars

[I 2021-02-16 19:35:14,718] Trial 1 finished with value: 0.6930163502693176 and parameters: {'nn_width': 160.0, 'nn_dropout': 0.45000000000000007, 'momentum': 0.08, 'virtual_batch_size': 10, 'weight_decay': -2, 'pct_start': 0.5}. Best is trial 0 with value: 0.6856903433799744.


...


best_valid_loss: 0.6852051615715027 - best_valid_metric: -2687.7641525505096
--------------------------------------------------------------------------------
sampled_params: {'nn_width': 144, 'dropout': 0.2, 'momentum': 0.06999999999999999, 'virtual_batch_size': 512, 'weight_decay': 1e-06, 'pct_start': 0.2}


[I 2021-02-16 23:49:39,036] Trial 15 finished with value: 0.6852724552154541 and parameters: {'nn_width': 144.0, 'nn_dropout': 0.2, 'momentum': 0.06999999999999999, 'virtual_batch_size': 9, 'weight_decay': -6, 'pct_start': 0.2}. Best is trial 10 with value: 0.6847517490386963.



best_valid_loss: 0.6852724552154541 - best_valid_metric: -2509.8445810300345
--------------------------------------------------------------------------------
sampled_params: {'nn_width': 112, 'dropout': 0.25, 'momentum': 0.09999999999999999, 'virtual_batch_size': 128, 'weight_decay': 0.0001, 'pct_start': 0.2}


[I 2021-02-17 00:10:55,904] Trial 16 finished with value: 0.6848864555358887 and parameters: {'nn_width': 112.0, 'nn_dropout': 0.25, 'momentum': 0.09999999999999999, 'virtual_batch_size': 7, 'weight_decay': -4, 'pct_start': 0.2}. Best is trial 10 with value: 0.6847517490386963.



best_valid_loss: 0.6848864555358887 - best_valid_metric: -2922.776335073502
--------------------------------------------------------------------------------
sampled_params: {'nn_width': 112, 'dropout': 0.2, 'momentum': 0.09, 'virtual_batch_size': 256, 'weight_decay': 1e-05, 'pct_start': 0.1}


***